In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Загрузка данных

In [2]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [4]:
data = pd.read_excel(
    'https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx?dl=1'
    ) 

#  Отбор признаков: мотивация

## Предобработка данных

In [5]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

## Обучение модели

In [6]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4682.957


## Удаление избыточного признака

In [7]:
x.drop('lat', axis = 1, inplace = True)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [9]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4672.930


#  Отбор признаков: классификация методов

## Метод рекурсивного исключения признаков

In [10]:
from sklearn.feature_selection import RFE

In [11]:
y = data['price']
x = data.drop(columns='price')

In [12]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [22]:
estimator = LinearRegression()
selectorRFE = RFE(estimator, n_features_to_select=3, step=1)
selectorRFE = selectorRFE.fit(X_train, y_train)
 
colsRFE = selectorRFE.get_feature_names_out()

In [14]:
X_train.columns

Index(['year', 'cylinders', 'odometer', 'lat', 'long', 'weather'], dtype='object')

In [19]:
selectorRFE.ranking_

array([1, 1, 4, 1, 3, 2])

##  МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

In [16]:
from sklearn.feature_selection import SelectKBest, f_regression

In [23]:
selectorSKB = SelectKBest(f_regression, k=3)
selectorSKB.fit(X_train, y_train)
 
colsSKB = selectorSKB.get_feature_names_out()

# Обучение модели на признаках отобранных RFE

In [30]:
# признаки от RFE и SKB селекторов
X_train_RFE = X_train[colsRFE]
X_test_RFE = X_test[colsRFE]
X_train_SKB = X_train[colsSKB]
X_test_SKB = X_test[colsSKB]

# класс модели линейной регрессии
linear_model = LinearRegression()

# обучение - предсказание для RFE
linear_model.fit(X_train_RFE, y_train)
y_predict_RFE = linear_model.predict(X_test_RFE)

# обучение - предсказание для SKB
linear_model.fit(X_train_SKB, y_train)
y_predict_SKB = linear_model.predict(X_test_SKB)

# Сравнение метрик
print(f'Показатель MAE для модели с отбором\
 признаков с помощью RFE: {round(mean_absolute_error(y_test, y_predict_RFE), 2)}')
print(f'Показатель MAE для модели с отбором\
 признаков с помощью SKB: {round(mean_absolute_error(y_test, y_predict_SKB), 2)}')

Показатель MAE для модели с отбором признаков с помощью RFE: 5096.57
Показатель MAE для модели с отбором признаков с помощью SKB: 4708.95


Показатель MAE для модели с отбором признаков с помощью SKB показал лучший результат на тестовой выборке равный **4708.95**